### OpenAI에서 지원하는 GPT API 사용 실습

In [4]:
!pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp38-cp38-win_amd64.whl.metadata (6.7 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp38-cp38-win_amd64.whl (2.0 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)


In [16]:
from openai import OpenAI
from getpass import getpass # 보안 정보를 화면에 표시 없이 입력받을 때 사용하는 모듈(api key 감추기용)

### API KEY 설정
- API key는 보안상 겉으로 보이지 않는 것이 좋으므로 감춰서 사용하자!

In [24]:
# 개인의 API key를 입력
MY_API_KEY = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [29]:
# OpenAI 객체 생성
client = OpenAI(api_key = MY_API_KEY)

### GPT 모델 불러와서 일반적인 질의

In [36]:
# gpt에게 입력할 텍스트
question = "안녕? 오늘 피크닉가기 좋은 날씨네!"
response = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                          messages = [{'role' : 'user', 'content':question}], # role은 user, system, assistant로 나뉨, system은 대전제
                                          max_tokens = 150,
                                          temperature = 0 # 0~2 낮을수록 간결하게(0.7이 디폴트)
                                         )

### ChatCompletion 객체에는 사용자의 요청과 모델의 응답에 대한 정보가 담겨져 있음
- content에 실제 텍스트 응답이 있으며, role에는 역할, model에는 사용된 모델명, usage에는 토큰 사용량에 대한 정보가 담겨져있음
- 모델이 응답할 때 role은 항상 assistant로 출력됨(우리가 질의할 때는 user로 질의해야 함)
- usage에 `prompt_tokens`는 입력 토큰(질의) 수, `completion_tokens`는 출력 토큰(응답)의 수가 됨

In [82]:
response

ChatCompletion(id='chatcmpl-Awk6kGCr87DcTeRVBNR7nbIjlynUB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='네, 맞아요! 피크닉 가기 딱 좋은 날이네요. 햇살도 따뜻하고 바람도 시원해서 즐거운 시간 보내시길 바래요. 좋은 하루 되세요!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738563626, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=74, prompt_tokens=34, total_tokens=108, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [84]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

In [80]:
response.choices[0].message.content

'네, 맞아요! 피크닉 가기 딱 좋은 날이네요. 햇살도 따뜻하고 바람도 시원해서 즐거운 시간 보내시길 바래요. 좋은 하루 되세요!'

### 자주 발생되는 에러
- RateLimitError : API 크레딧 금액 한도 초과
- Invalid RequetError : 잘못된 요청
- APIConnectionError : API 서버와의 연결 문제로 발생
- OpenAIError : 일반적인 openai 라이브러리 사용시 발생되는 에러

### 프로그래밍 기술적인 내용 질의

In [ ]:
# gpt에게 입력할 텍스트
question = "Pandas DataFrame에서 중복 항목을 제거하는 방법은?"
response = client.chat.completions.create(model = 'gpt-3.5-turbo',
                                          messages = [{'role' : 'user', 'content':question}],
                                          max_tokens = 150,
                                         )

In [95]:
print(response.choices[0].message.content)

Pandas DataFrame에서 중복 항목을 제거하기 위해서는 `drop_duplicates()` 메소드를 사용하면 됩니다. 이 메소드는 중복된 행을 제거하여 유일한 값만을 남기고 새로운 DataFrame을 반환합니다. 

예를 들어, 다음과 같이 DataFrame에서 중복 항목을 제거할 수 있습니다.

```python
import pandas as pd

# 샘플 데이터프레임 생성
data = {'A': [1, 2, 2, 3, 4],
        'B': ['x', 'y', 'y', '


### 입력한 파일에 대한 분석 요청
- 파일과 질의를 구분해서 사용자 정의 함수 만들기

In [104]:
def anlysis_data(data, question, model):
    prompt = f"Data: {data}\nQuestion : {question}"
    response = client.chat.completions.create(model = model,
                                              messages = [{'role':'user','content':prompt}],
                                              max_tokens = 500
                                             )
    return response.choices[0].message.content

In [108]:
data = "data/bmi_500.csv"
question = "해당 데이터는 500명의 bmi점수야. 저체중부터 고도비만까지 데이터의 개수를 파악해서 출력해줘."
model = "gpt-3.5-turbo"

result = anlysis_data(data, question, model)
print(result)

해당 데이터에서는 다음과 같이 BMI 범주별 데이터 개수가 나타납니다:

- 저체중: 76명
- 정상: 183명
- 과체중: 134명
- 비만: 107명
- 고도비만: 0명


### 정리
- API를 활용하여 로컬 PC에서 결과를 출력하려 한다면 사용자별로 환경이 다르기 때문에 시각화 요청에 대해 누구나 활용할 수 있는 형태로 제공됨
- ChatGPT 웹 서비스에서는 3.5-turbo 버전에서 파일 입력이 불가하지만, 3.5-turbo API를 호출하여 사용하는 경우 로컬 PC 파일을 입력할 수 있음